In [16]:
# ==================== ONE-CELL YOLO11 VEHICLE + LICENSE PLATE PROJECT ====================

# ---------------------------------------------
# INSTALL PACKAGES
# ---------------------------------------------
!pip install ultralytics opencv-python easyocr matplotlib --quiet

# ---------------------------------------------
# IMPORT LIBRARIES
# ---------------------------------------------
from ultralytics import YOLO
import cv2, os, shutil
import matplotlib.pyplot as plt
import easyocr
import numpy as np

# ---------------------------------------------
# TRAINING SETUP (YOU CAN SKIP IF ALREADY TRAINED)
# ---------------------------------------------
# DATASET STRUCTURE REQUIREMENT (YOLO FORMAT)
# data.yaml MUST EXIST IN THIS FORMAT:
# train: datasets/images/train
# val: datasets/images/val
# nc: 2
# names: ["vehicle", "license_plate"]

TRAIN_MODEL = False  # TRUE = Train New Model, FALSE = Use Pretrained

if TRAIN_MODEL:
    model = YOLO("yolo11n.pt")  # base model
    model.train(
        data="data.yaml",  
        epochs=10,         # increase for real training
        imgsz=640,
        batch=8,
        name="vehicle_lp_model"
    )
    model_path = "runs/detect/vehicle_lp_model/weights/best.pt"
else:
    # pretrained or your trained model path
    model_path = "yolo11n.pt"   # change to best.pt

model = YOLO(model_path)

# ---------------------------------------------
# LICENSE PLATE CROPPER
# ---------------------------------------------
def crop_license_plates(frame, results, lp_class_id=1):
    crops = []
    for box in results[0].boxes:
        cls = int(box.cls[0])
        if cls == lp_class_id:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            crops.append(frame[y1:y2, x1:x2])
    return crops

# ---------------------------------------------
# OCR READER
# ---------------------------------------------
reader = easyocr.Reader(['en'])

def read_plate(crop):
    if crop.size == 0:
        return ""
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    res = reader.readtext(gray)
    if len(res) > 0:
        return res[0][1]
    return ""

# ---------------------------------------------
# IMAGE DETECTION PIPELINE
# ---------------------------------------------
def detect_image(img_path):
    img = cv2.imread(img_path)
    results = model(img)

    # show detection
    plt.figure(figsize=(10, 8))
    plt.imshow(results[0].plot())
    plt.axis("off")
    plt.title("YOLO11 Vehicle + License Plate Detection")
    plt.show()

    # crop + ocr
    lp_crops = crop_license_plates(img, results)
    for i, crop in enumerate(lp_crops):
        text = read_plate(crop)
        plt.figure(figsize=(5, 4))
        plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        plt.title(f"Plate {i+1}: {text}")
        plt.axis("off")
        plt.show()

    print("OCR RESULT:", [read_plate(c) for c in lp_crops])

# ---------------------------------------------
# VIDEO DETECTION PIPELINE
# ---------------------------------------------
def detect_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        annotated = results[0].plot()
        
        cv2.imshow("YOLO11 Vehicle + Plate - VIDEO", annotated)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
            
    cap.release()
    cv2.destroyAllWindows()

# ---------------------------------------------
# WEBCAM DETECTION PIPELINE
# ---------------------------------------------
def detect_webcam():
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        annotated = results[0].plot()

        cv2.imshow("YOLO11 Vehicle + Plate - WEBCAM", annotated)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
            
    cap.release()
    cv2.destroyAllWindows()

# ---------------------------------------------
# RUN EXAMPLES
# ---------------------------------------------

# 1️⃣ IMAGE DETECTION
# detect_image("car.jpg")

# 2️⃣ VIDEO DETECTION
# detect_video("traffic.mp4")

# 3️⃣ WEBCAM LIVE DETECTION
# detect_webcam()

print("ONE-CELL YOLO PROJECT READY ✔")


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\mayur\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

ONE-CELL YOLO PROJECT READY ✔████████████████████████████████| 100.0% Complete


In [17]:
detect_image("car.jpg")


FileNotFoundError: [Errno 2] No such file or directory: 'car.jpg'